In [1]:
import geemap
import ee

Map = geemap.Map(center=[30, 112])
Map

Enter verification code:  4/1AeaYSHCFN1adaTGhEszzhU2eHk4uDJ47trMdeywUTlWaF4POFeDwBNaMWLg



Successfully saved authorization token.


Map(center=[30, 112], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [2]:
import os
os.chdir(r'D:\Data\parper\data')

In [3]:
shp = geemap.shp_to_ee('shp/JX.shp')
roi = shp.geometry()
Map.center_object(roi, 8)
Map.addLayer(roi, {}, 'roi')

In [ ]:
points = geemap.shp_to_ee('shp/landuse_point.shp')

Map.addLayer(points, {'color': 'FF0000'}, 'points')

In [ ]:
from Monarch import user_gee
year = 2018
s2_year = []
lswis = []
ndvis = []
evis = []
for i in range(1, 13):
    s2 = ee.ImageCollection("COPERNICUS/S2")\
            .filterBounds(roi)\
            .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
            .filterDate(str(year), str(year+1))\
            .filter(ee.Filter.calendarRange(i, i, 'month'))
    s2_rm_cloud = s2.map(user_gee.rm_cloud_s2_sr)
    s2_z = s2_rm_cloud.merge(s2).mosaic().select(['B2', 'B3', 'B4', 'B5', 'B8', 'B11'])
    a = s2.aggregate_array('system:time_start').map(lambda i: ee.Date(i).format('YYYY-MM-dd'))
    evi = s2_z.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': s2_z.select('B8'),
        'RED': s2_z.select('B4'),
        'BLUE': s2_z.select('B2')
    }).rename("S2_EVI")
    ndvi = s2_z.normalizedDifference(['B8', 'B4']).rename("S2_NDVI")
    lswi = s2_z.normalizedDifference(['B8', 'B11']).rename("S2_LSWI")
    ndvis.append(ndvi)
    lswis.append(lswi)
    evis.append(evi)
    s2_image = s2_z.addBands(ndvi)
    s2_image = s2_image.addBands(lswi)
    # s2_image = s2_image.addBands(evi)
    s2_year.append(s2_image)
    break

In [ ]:
Map.addLayer(s2_rm_cloud.select('B2'), {}, 's2_rm_mo')

In [ ]:
Map.addLayer(s2_rm_cloud.select('B2').mean(), {}, 's2_rm_mean')

In [ ]:
Map

In [ ]:
image = ee.ImageCollection(s2_year).toBands().clip(roi)

In [ ]:
lswi = ee.ImageCollection(lswis).toBands().clip(roi)
ndvi = ee.ImageCollection(ndvis).toBands().clip(roi)

In [ ]:
com1 = ndvi.expression('gt1 && gt2', {
    'gt1': ndvi.select('7_S2_NDVI').gt(0.35),
    'gt2': ndvi.select('8_S2_NDVI').gt(0.35)
}).rename('ndvi_com')
com2 = lswi.expression('gt1 && lt2', {
    'gt1': lswi.select('8_S2_LSWI').gt(0.15),
    'lt2': lswi.select('10_S2_LSWI').lt(0.15)
}).rename('lswi_com')
image = image.addBands(com1)
image = image.addBands(com2)

In [ ]:
ps = points.filter(ee.Filter.lt('CID', 700))
ps_y = points.filter(ee.Filter.gte('CID', 700))

In [ ]:
label = 'TDLYDM'

training = image.sampleRegions(**{
  'collection': ps,
  'properties': [label],
  'scale': 10
})
# 使用随机森林分类
trained = ee.Classifier.smileRandomForest(numberOfTrees=100).train(training, label)

In [ ]:
result = image.classify(trained)

geemap.ee_export_image_to_asset(result, 'JX_class_3', 'JX_class_3', scale=10, crs='epsg:3857', region=roi, maxPixels=1e10)

In [ ]:
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

In [ ]:
landuse = ee.Image('users/2431566134liumonarch/JX_class_3')

In [ ]:
Map.addLayer(landuse.randomVisualizer(), {}, 'classfied')
Map

In [ ]:
label = 'TDLYDM'

verify = landuse.sampleRegions(**{
  'collection': ps_y,
  'properties': [label],
  'scale': 10
})

In [ ]:
path_csv = 'verify_ps_2017.csv'
geemap.zonal_statistics(landuse, ps_y, path_csv, scale=10)

In [ ]:
geemap.ee_export_image_to_drive(landuse, 'JX_class', 'Image', region=roi, crs='epsg:3857', scale=10, maxPixels=1e10)

In [ ]:
landuse_rice = landuse.mask(landuse.eq(3))
# Map.addLayer(landuse_rice, {'palette': 'red', 'max':3, 'min':3}, 'rich')

In [ ]:
landuse_count = landuse_rice.connectedPixelCount(50, False)

In [ ]:
landuse_make = landuse.mask(landuse_count.gte(10))
landuse_make_a = landuse_make.unmask(1)
Map.addLayer(landuse_make, {'max':3, 'min':0}, 'make')

In [ ]:
geemap.ee_export_image_to_drive(landuse_make, 'JX_rice', 'Image', region=roi, crs='epsg:3857', scale=10, maxPixels=1e10)

In [ ]:
path_csv = 'verify_ps_2017_1.csv'
geemap.zonal_statistics(landuse_make_a, ps_y, path_csv, scale=10)

In [ ]:
a = landuse_rice.reduceRegion(ee.Reducer.sum(), geometry=roi, scale=10, crs='epsg:3857', maxPixels=1e10)

In [ ]:
a.getInfo()

In [ ]:
790888352 / 3